In [2]:
# !python -m pip install websockets
# !python -m pip install websocket-client
!python -m pip install "python-socketio[client]==5.8.0"



  Attempting uninstall: python-socketio
    Found existing installation: python-socketio 5.11.4
    Uninstalling python-socketio-5.11.4:
      Successfully uninstalled python-socketio-5.11.4


In [4]:
import asyncio
import websockets
import json

async def test_chemistry3d_connection():
    # Define the WebSocket URI for the Chemistry3D client ("/chemistry3d" path)
    websocket_uri = "ws://localhost:8081/chemistry3d"  # Replace PORT with the actual port of the relay server

    # Connect to the WebSocket server
    async with websockets.connect(websocket_uri) as websocket:
        print("Connected to the relay server at /chemistry3d.")

        # Send a test message to the server
        test_message = {
            "type": "test_message",
            "content": "Hello, Chemistry3D Relay Server!"
        }
        await websocket.send(json.dumps(test_message))
        print("Sent test message:", test_message)

        # Listen for a response from the server
        response = await websocket.recv()
        print("Received response:", response)

# Run the test in the Jupyter Notebook
await test_chemistry3d_connection()


Connected to the relay server at /chemistry3d.
Sent test message: {'type': 'test_message', 'content': 'Hello, Chemistry3D Relay Server!'}
Received response: {"source":"chemistry3d","type":"received","data":{"event":{"type":"test_message","content":"Hello, Chemistry3D Relay Server!"}},"timestamp":"2024-11-16T00:40:04.039Z"}


In [3]:
import asyncio
import websockets
import json

async def test_relay_connection():
    # Define the WebSocket URI for the React client ("/ws" path)
    websocket_uri = "ws://localhost:8081/ws"  # Replace PORT with the actual port of the relay server

    # Connect to the WebSocket server
    async with websockets.connect(websocket_uri) as websocket:
        print("Connected to the relay server.")

        # Send a test message to the server
        test_message = {
            "type": "test_message",
            "content": "Hello, Relay Server!"
        }
        await websocket.send(json.dumps(test_message))
        print("Sent test message:", test_message)

        # Listen for a response from the server
        response = await websocket.recv()
        print("Received response:", response)

# Run the test in the Jupyter Notebook
await test_relay_connection()


Connected to the relay server.
Sent test message: {'type': 'test_message', 'content': 'Hello, Relay Server!'}
Received response: {"type":"status","isConnected":true,"connectedClients":1,"chemistry3dConnected":false,"timestamp":"2024-11-16T00:38:02.573Z"}


In [26]:
import socketio
import threading
import asyncio
import json
from IPython.display import display, Javascript

# Initialize Socket.IO client
sio = socketio.Client(logger=True, engineio_logger=True)
websocket_connected = False

# Message queue for incoming function calls
tool_calls_queue = []

def print_and_send(message):
    """Helper function to print and send messages to the WebSocket server."""
    print(message)
    if websocket_connected:
        try:
            sio.emit('message', json.dumps({'text': message}))
        except Exception as e:
            print(f'Error sending message to WebSocket server: {e}')

# Define event handlers
@sio.event
def connect():
    global websocket_connected
    print('WebSocket client connected to the relay server')
    websocket_connected = True

@sio.event
def disconnect():
    global websocket_connected
    print('WebSocket client disconnected from the relay server')
    websocket_connected = False

@sio.on('function_call')
def on_function_call(data):
    """Handle incoming function calls."""
    print(f'Received function_call from relay server: {data}')
    tool_calls_queue.append(data)

@sio.on('log')
def on_log(data):
    """Handle logs from the relay server."""
    print(f"Relay server log: {data}")

# Connect to the WebSocket server
try:
    # Change the port if the server is running on a different port
    sio.connect('http://localhost:8081', transports=['websocket'], wait=True)
except Exception as e:
    print(f'Failed to connect to relay server: {e}')

# Function to send a user prompt from the Jupyter Notebook input
def send_user_prompt(prompt):
    if websocket_connected:
        print_and_send(f'Sending user prompt to relay server: {prompt}')
    else:
        print('WebSocket is not connected. Cannot send user input.')

# Emulate user input and function call processing in a Jupyter Notebook
async def emulate_client_behavior():
    while True:
        # Emulate checking for function calls in the queue
        if tool_calls_queue:
            function_call = tool_calls_queue.pop(0)
            try:
                print(f"Processing function call: {function_call}")
                # Simulate processing the function call and prepare a response
                result = {"status": "success", "message": "Function executed successfully"}
                # Send the result back to the relay server
                sio.emit('function_call_output', {
                    'call_id': function_call.get('id', ''),
                    'output': result
                })
                print(f"Function call result sent: {result}")
            except Exception as e:
                print(f"Error processing function_call: {e}")

        await asyncio.sleep(1)  # Simulate a wait time for event handling

# Start the event loop in a Jupyter Notebook
loop = asyncio.get_event_loop()
if loop.is_running():
    display(Javascript('Jupyter.notebook.kernel.restart()'))
else:
    threading.Thread(target=lambda: asyncio.run(emulate_client_behavior()), daemon=True).start()

# Send a test prompt from the Notebook
send_user_prompt("Start a chemical mixing task")


Attempting WebSocket connection to ws://localhost:8081/socket.io/?transport=websocket&EIO=4


Failed to connect to relay server: Connection error


<IPython.core.display.Javascript object>

WebSocket is not connected. Cannot send user input.


In [18]:
# Emulate Chemistry3D client in Jupyter Notebook using Socket.IO

import socketio
import time
from IPython.display import display, Markdown

# Create a Socket.IO client
sio = socketio.Client(logger=True, engineio_logger=True)

# Define event handlers
@sio.event
def connect():
    print("Connected to the relay server")
    display(Markdown("**Connected to the relay server**"))

@sio.event
def connect_error(data):
    print(f"Connection failed: {data}")
    display(Markdown(f"**Connection failed: {data}**"))

@sio.event
def disconnect():
    print("Disconnected from the relay server")
    display(Markdown("**Disconnected from the relay server**"))

@sio.on('function_call')
def on_function_call(data):
    print(f"Received function_call: {data}")
    display(Markdown(f"**Received function_call:** {data}"))

@sio.on('log')
def on_log(data):
    print(f"Log from server: {data}")
    display(Markdown(f"**Log from server:** {data}"))

# Connect to the relay server
try:
    sio.connect(
        'http://localhost:8081',  # Replace with the correct server address if different
        namespaces=['/chemistry3d'],  # Connect to the specific namespace
        transports=['websocket'],
        socketio_path='/chemistry3d/socket.io'  # Custom path to match the server's path
    )
    print("Connection established")
except Exception as e:
    print(f"Failed to connect: {e}")

# Function to send a message to the relay server
def send_message_to_server(message):
    print(f"Sending message: {message}")
    sio.emit('message', message, namespace='/chemistry3d')

# Send a test message to the server
send_message_to_server("Hello from the Chemistry3D client emulation!")

# Keep the connection open for testing
try:
    while True:
        time.sleep(1)  # Keep the script running to listen for server responses
except KeyboardInterrupt:
    print("Stopping client...")
    sio.disconnect()


Attempting WebSocket connection to ws://localhost:8081/chemistry3d/socket.io/?transport=websocket&EIO=4


Failed to connect: Connection error
Sending message: Hello from the Chemistry3D client emulation!


BadNamespaceError: /chemistry3d is not a connected namespace.

In [11]:
# test_socketio_client.py

import socketio

sio = socketio.Client(logger=True, engineio_logger=True)

@sio.event
def connect():
    print('Connection established')

@sio.event
def connect_error(data):
    print('Connection failed')

@sio.event
def disconnect():
    print('Disconnected from server')

try:
    sio.connect('http://localhost:8081/chemistry3d', transports=['websocket'])
    sio.wait()
except Exception as e:
    print(f'Failed to connect: {e}')


Attempting WebSocket connection to ws://localhost:8081/socket.io/?transport=websocket&EIO=4


Connection failed
Failed to connect: Connection error


In [19]:
# test_socketio_client.py

import socketio

sio = socketio.Client(logger=True, engineio_logger=True)

@sio.event
def connect():
    print('Connection established')

@sio.event
def connect_error(data):
    print('Connection failed')

@sio.event
def disconnect():
    print('Disconnected from server')

try:
    sio.connect('http://localhost:8081', transports=['websocket'])
    sio.wait()
except Exception as e:
    print(f'Failed to connect: {e}')


Attempting WebSocket connection to ws://localhost:8081/socket.io/?transport=websocket&EIO=4


Connection failed
Failed to connect: Connection error


In [15]:
import websocket
import json
import threading
import time

def on_message(ws, message):
    print(f"Received message from server: {message}")

def on_error(ws, error):
    print(f"Encountered error: {error}")

def on_close(ws, close_status_code, close_msg):
    print("WebSocket connection closed")

def on_open(ws):
    print("WebSocket connection opened")
    # Send message after connection is established
    tool_calls = [
        {
            "function_name": "load_molecule",
            "arguments": {
                "molecule": "H2O"
            }
        }
    ]
    tool_calls_json = json.dumps(tool_calls)
    ws.send(tool_calls_json)

# Create the WebSocket app
ws_url = "ws://localhost:8765"
ws_app = websocket.WebSocketApp(
    ws_url,
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

# Run the WebSocket app in a separate thread
def run_ws():
    ws_app.run_forever()

ws_thread = threading.Thread(target=run_ws)
ws_thread.daemon = True
ws_thread.start()

# Wait for the connection to establish
time.sleep(2)  # Adjust if necessary

# Optionally, send messages here if not sent in on_open


Encountered error: [Errno 111] Connection refused
WebSocket connection closed


In [10]:
import websocket
import json
import threading

def on_message(ws, message):
    print(f"Received message from server: {message}")

def on_error(ws, error):
    print(f"Encountered error: {error}")

def on_close(ws, close_status_code, close_msg):
    print("WebSocket connection closed")

def on_open(ws):
    print("WebSocket connection opened")

# Create the WebSocket app
ws_url = "ws://localhost:8765"
ws_app = websocket.WebSocketApp(
    ws_url,
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

# Run the WebSocket app in a separate thread
def run_ws():
    ws_app.run_forever()

ws_thread = threading.Thread(target=run_ws)
ws_thread.daemon = True
ws_thread.start()

# Wait briefly to ensure the connection is open
import time
time.sleep(1)

# Define and send the tool_calls message
tool_calls = [
    {
        "function_name": "load_molecule",
        "arguments": {
            "molecule": "H2O"
        }
    },
    {
        "function_name": "mix_solutions",
        "arguments": {
            "solution1": "beaker_Kmno4",
            "solution2": "beaker_Fecl2"
        }
    }
]

tool_calls_json = json.dumps(tool_calls)
ws_app.send(tool_calls_json)

# Optional: Keep the notebook cell running to receive messages
# time.sleep(10)  # Adjust the sleep time as needed

# Close the WebSocket connection when done
# ws_app.close()


Encountered error: [Errno 111] Connection refused
WebSocket connection closed


<frozen genericpath>:77: RuntimeWarning: coroutine 'test_websocket' was never awaited


WebSocketConnectionClosedException: Connection is already closed.

In [23]:
import asyncio
import websockets

async def test_websocket():
    uri = "ws://localhost:8081"
    async with websockets.connect(uri) as websocket:
        await websocket.send("Hello, server")
        response = await websocket.recv()
        print(f"Received from server: {response}")

await test_websocket()

InvalidStatusCode: server rejected WebSocket connection: HTTP 400